In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 26.5 MB/s 
     |████████████████████████████████| 77 kB 7.4 MB/s 
     |████████████████████████████████| 895 kB 60.4 MB/s 
     |████████████████████████████████| 6.6 MB 63.2 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 29.3 MB/s 
     |████████████████████████████████| 1.1 MB 58.5 MB/s 
     |████████████████████████████████| 136 kB 74.9 MB/s 
     |████████████████████████████████| 212 kB 71.6 MB/s 
     |████████████████████████████████| 127 kB 75.3 MB/s 
     |████████████████████████████████| 271 kB 73.7 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 144 kB 72.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import ast

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/NLP_Project/data_with_keywords_train.csv')
data

,Unnamed: 0,Unnamed: 0.1,title,body,num_comments,verdict,word_count,has_tldr,has_background,has_update,has_edit,keywords,title_key_word
0,1881,1881,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",43.0,not the asshole,573,False,False,False,True,"['hospital', 'drive', 'boyfriend', 'asking', '...","['aita asking', 'boyfriend drive', 'hospital',..."
1,33358,33358,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,24.0,not the asshole,267,False,False,False,False,"['close friend ’', 'ex boyfriend', 'phone', 'b...","['buying phone', 'aita', 'friend', 'ex boyfrie..."
2,36512,36512,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,20.0,asshole,232,False,False,False,False,"['sleepover', 'phone', 'mom', 'answering', 'ai...","['mom sleepover', 'aita answering', 'phone', '..."
3,28733,28733,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,58.0,asshole,418,False,False,False,False,"['straighten', 'hair', 'girlfriend', 'asking',...","['girlfriend straighten', 'aita asking', 'hair..."
4,23625,23625,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",1164.0,asshole,280,False,False,False,False,"['stop eating “ baby snacks ”?', 'telling', 'g...","['baby snacks', 'aita telling', 'girlfriend st..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37507,19883,19883,aita for letting someone sit in my seat at the...,i went to watch a play with my girlfriend last...,86.0,asshole,507,False,False,False,False,"['letting someone sit', 'theatre', 'seat', 'ai...","['seat theatre', 'letting', 'aita', 'sit', 'se..."
37508,14730,14730,aita for venting in a kind of nasty way about ...,so i was hanging out with my friend and i star...,7.0,asshole,229,False,False,False,False,"['nasty way', 'venting', 'kind', 'classmates',...","['aita venting', 'way classmates', 'kind nasty..."
37509,4116,4116,wibta if i asked my neighbor to help with the ...,tl;dr at the bottom.\n\n\n**i will also likely...,56.0,asshole,463,False,False,False,True,"['wibta', 'tree', 'removing', 'neighbor', 'hom...","['neighbor help', 'wibta asked', 'cost removin..."
37510,14580,14580,aita for not wanting my fiancée to invite her ...,"my fiancée and i have been dating 3 years, eng...",115.0,asshole,222,False,False,False,False,"['wedding', 'wanting', 'stepdaughter', 'invite...","['stepdaughter wedding', 'invite', 'aita', 'wa..."


In [ ]:
type(data.iloc[0].keywords)

str

In [ ]:
data_truncated = data[data['verdict'].isin(['asshole', 'not the asshole'])]
len(data_truncated)

37512

In [ ]:
data_truncated.head()

,Unnamed: 0,Unnamed: 0.1,title,body,num_comments,verdict,word_count,has_tldr,has_background,has_update,has_edit,keywords,title_key_word
0,1881,1881,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",43.0,not the asshole,573,False,False,False,True,"['hospital', 'drive', 'boyfriend', 'asking', '...","['aita asking', 'boyfriend drive', 'hospital',..."
1,33358,33358,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,24.0,not the asshole,267,False,False,False,False,"['close friend ’', 'ex boyfriend', 'phone', 'b...","['buying phone', 'aita', 'friend', 'ex boyfrie..."
2,36512,36512,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,20.0,asshole,232,False,False,False,False,"['sleepover', 'phone', 'mom', 'answering', 'ai...","['mom sleepover', 'aita answering', 'phone', '..."
3,28733,28733,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,58.0,asshole,418,False,False,False,False,"['straighten', 'hair', 'girlfriend', 'asking',...","['girlfriend straighten', 'aita asking', 'hair..."
4,23625,23625,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",1164.0,asshole,280,False,False,False,False,"['stop eating “ baby snacks ”?', 'telling', 'g...","['baby snacks', 'aita telling', 'girlfriend st..."


In [ ]:
data_truncated = data_truncated[['title', 'body', 'verdict', 'keywords']]

In [ ]:
data_truncated = data_truncated.fillna('')
data_truncated

,title,body,verdict,keywords
0,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",not the asshole,"['hospital', 'drive', 'boyfriend', 'asking', '..."
1,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,not the asshole,"['close friend ’', 'ex boyfriend', 'phone', 'b..."
2,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,asshole,"['sleepover', 'phone', 'mom', 'answering', 'ai..."
3,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,asshole,"['straighten', 'hair', 'girlfriend', 'asking',..."
4,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",asshole,"['stop eating “ baby snacks ”?', 'telling', 'g..."
...,...,...,...,...
37507,aita for letting someone sit in my seat at the...,i went to watch a play with my girlfriend last...,asshole,"['letting someone sit', 'theatre', 'seat', 'ai..."
37508,aita for venting in a kind of nasty way about ...,so i was hanging out with my friend and i star...,asshole,"['nasty way', 'venting', 'kind', 'classmates',..."
37509,wibta if i asked my neighbor to help with the ...,tl;dr at the bottom.\n\n\n**i will also likely...,asshole,"['wibta', 'tree', 'removing', 'neighbor', 'hom..."
37510,aita for not wanting my fiancée to invite her ...,"my fiancée and i have been dating 3 years, eng...",asshole,"['wedding', 'wanting', 'stepdaughter', 'invite..."


In [ ]:
print(data_truncated.iloc[37511].title)
print(data_truncated.iloc[37511].keywords)

aita for being disrespectful with my mum over her hating anime?
['hating anime', 'mum', 'disrespectful', 'aita']


In [ ]:
TOKENS_THRESHOLD = 200
def filter_fn(row):
    if len(row['body'].split()) > TOKENS_THRESHOLD:
        return True
    else:
        return False

data_truncated = data_truncated[data_truncated.apply(filter_fn, axis=1)]
data_truncated = data_truncated.reset_index()
data_truncated

,index,title,body,verdict,keywords
0,0,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",not the asshole,"['hospital', 'drive', 'boyfriend', 'asking', '..."
1,1,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,not the asshole,"['close friend ’', 'ex boyfriend', 'phone', 'b..."
2,2,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,asshole,"['sleepover', 'phone', 'mom', 'answering', 'ai..."
3,3,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,asshole,"['straighten', 'hair', 'girlfriend', 'asking',..."
4,4,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",asshole,"['stop eating “ baby snacks ”?', 'telling', 'g..."
...,...,...,...,...,...
29328,37507,aita for letting someone sit in my seat at the...,i went to watch a play with my girlfriend last...,asshole,"['letting someone sit', 'theatre', 'seat', 'ai..."
29329,37508,aita for venting in a kind of nasty way about ...,so i was hanging out with my friend and i star...,asshole,"['nasty way', 'venting', 'kind', 'classmates',..."
29330,37509,wibta if i asked my neighbor to help with the ...,tl;dr at the bottom.\n\n\n**i will also likely...,asshole,"['wibta', 'tree', 'removing', 'neighbor', 'hom..."
29331,37510,aita for not wanting my fiancée to invite her ...,"my fiancée and i have been dating 3 years, eng...",asshole,"['wedding', 'wanting', 'stepdaughter', 'invite..."


In [ ]:
data_small = data_truncated.sample(n=25000)
data_small

,index,title,body,verdict,keywords
23242,29722,aita for trying to bring christmas equality,throw away because i do feel a bit guilty - \n...,asshole,"['bring christmas equality', 'trying', 'aita']"
1987,2561,aita the asshole for not wanting to apologise ...,"for context, this is in the military. every bu...",asshole,"['wanting', 'bunkmate', 'asshole', 'apologise'..."
11477,14718,aita for leaving my brothers birthday dinner b...,gonna try to keep it short without missing any...,not the asshole,"['brothers birthday dinner', 'didnt want', 'pa..."
12843,16458,aita for getting mad at my mum for not listening.,so a few months ago my mum and dad found out t...,asshole,"['getting mad', 'mum', 'listening', 'aita']"
7422,9556,aita for confronting a role-player about tryin...,so we have a dnd large role-playing group on d...,not the asshole,"['make everything', 'trying', 'role', 'player'..."
...,...,...,...,...,...
3422,4385,aita for delaying breaking up with my gf?,"hello there r/aita, long time lurker, first ti...",asshole,"['delaying breaking', 'gf', 'aita']"
15836,20257,aita for getting frustrated with a friend who ...,"for clarification, these mobility issues are n...",not the asshole,"['mobility issues', 'getting frustrated', 'fri..."
26850,34332,aita for not slowing down baseball with my son...,i (40s/m) have one son (14/m) and a step daugh...,asshole,"['stepdaughter', 'son', 'slowing', 'baseball',..."
22003,28135,aita for getting pissed at a newly made friend...,i take pictures and videos for other people fo...,asshole,"['newly made friend', 'getting pissed', 'uploa..."


In [ ]:
data_small = data_small.reset_index(drop = True)
data_small = data_small[['title', 'body', 'verdict', 'keywords']]
data_small

,title,body,verdict,keywords
0,aita for trying to bring christmas equality,throw away because i do feel a bit guilty - \n...,asshole,"['bring christmas equality', 'trying', 'aita']"
1,aita the asshole for not wanting to apologise ...,"for context, this is in the military. every bu...",asshole,"['wanting', 'bunkmate', 'asshole', 'apologise'..."
2,aita for leaving my brothers birthday dinner b...,gonna try to keep it short without missing any...,not the asshole,"['brothers birthday dinner', 'didnt want', 'pa..."
3,aita for getting mad at my mum for not listening.,so a few months ago my mum and dad found out t...,asshole,"['getting mad', 'mum', 'listening', 'aita']"
4,aita for confronting a role-player about tryin...,so we have a dnd large role-playing group on d...,not the asshole,"['make everything', 'trying', 'role', 'player'..."
...,...,...,...,...
24995,aita for delaying breaking up with my gf?,"hello there r/aita, long time lurker, first ti...",asshole,"['delaying breaking', 'gf', 'aita']"
24996,aita for getting frustrated with a friend who ...,"for clarification, these mobility issues are n...",not the asshole,"['mobility issues', 'getting frustrated', 'fri..."
24997,aita for not slowing down baseball with my son...,i (40s/m) have one son (14/m) and a step daugh...,asshole,"['stepdaughter', 'son', 'slowing', 'baseball',..."
24998,aita for getting pissed at a newly made friend...,i take pictures and videos for other people fo...,asshole,"['newly made friend', 'getting pissed', 'uploa..."


In [ ]:
data_small['verdict'].value_counts()

not the asshole    12968
asshole            12032
Name: verdict, dtype: int64

In [ ]:
data_truncated['verdict'].value_counts()

not the asshole    15200
asshole            14133
Name: verdict, dtype: int64

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', 
                                          bos_token='<|startofprompt|>', 
                                          eos_token='<|endoftext|>', 
                                          pad_token='<|pad|>',
                                         )


tokenizer.encode("Sample Text")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[36674, 8255]

In [ ]:
len(tokenizer)

50259

In [ ]:
tokenizer.add_tokens(['<|endofprompt|>', '<|startofverdict|>', '<|endofverdict|>', '<|sok|>', '<|eok|>', '<|startoftext|>'], special_tokens=True)
tokenizer.convert_tokens_to_ids(['<|endofprompt|>', '<|startofverdict|>', '<|endofverdict|>', '<|sok|>', '<|eok|>', '<|startoftext|>'])

[50259, 50260, 50261, 50262, 50263, 50264]

In [ ]:
tokenizer.all_special_tokens

['<|startofprompt|>', '<|endoftext|>', '<|pad|>']

In [ ]:
len(tokenizer)

50265

In [ ]:
titles = data_small.title.copy()
verdicts = data_small.verdict.copy()
stories = data_small.body.copy()
keywords = data_small.keywords.copy()

keywords = pd.Series(list(map(lambda x: ast.literal_eval(x), keywords)))
keywords = list(map(lambda x: join_keywords(x), keywords))

In [ ]:
sample_title = data_small[:1]['title'].values[0]
sample_verdict = data_small[:1]['verdict'].values[0]
sample_body = data_small[:1]['body'].values[0]
# sample_keywords = data_small[:1]['keywords'].values[0]
sample_keywords = keywords[0]
print(sample_title)
print(sample_verdict)
print(sample_body)
print(sample_keywords)

aita for trying to bring christmas equality
asshole
throw away because i do feel a bit guilty - 

we have had a very eventful year (bought a house in june, had twins in july) which has led us to being a little lower on cash then usual. now that we're coming up on christmas i am trying to keep us on budget but not make our older kids feel completely stiffed (i usually try to buy them what they asked for)

here's the situation, i have a son and stepdaughter (both 11 yo). every year my stepdaughters grandmother goes above and beyond for christmas. she buys things for both of the kids but every year my stepdaughter has a big "surprise" box of things they sneak off to stores and buy for my stepdaughter. it's usually a bunch of toys and clothes that my stepdaughter has picked out for herself and she opens on christmas day. my mother in law insists she "spends the same amount on everybody" but the things in the box are usually worth $200 or more. this annoys me every year because they lie abo

In [ ]:
def join_keywords(keywords, randomize=True):
    N = len(keywords)

    #random sampling and shuffle
    if randomize: 
        # M = random.choice(range(N+1))
        keywords = keywords[:N]
        random.shuffle(keywords)

    return ','.join(keywords)

In [ ]:
len(tokenizer.encode("<|startofprompt|>" + sample_title + "<|endofprompt|>" + sample_verdict + "<|endofverdict|>" + sample_keywords + "<|eok|>"+ sample_body + "<|endoftext|>"))

412

In [ ]:
bs = 1
max_story = 800

In [ ]:
import torch
torch.manual_seed(67)
from torch.utils.data import Dataset 

class AitaDataset(Dataset):

  def __init__(self, titles, verdicts, txt_list, tokenizer, gpt2_type="gpt2", max_length=max_story):

    self.tokenizer = tokenizer # the gpt2 tokenizer we instantiated
    self.input_ids = []
    self.attn_masks = []
    self.labels = []

    for i in range(len(txt_list)):
      """
      This loop will iterate through each entry in the text corpus.
      For each bit of text it will prepend it with the start of text token,
      then append the end of text token and pad to the maximum length with the 
      pad token. 
      """

      encodings_dict = tokenizer('<|startofprompt|>'+ titles[i] + "<|endofprompt|>" 
                                 + verdicts[i] + "<|endofverdict|>"
                                 + keywords[i] + "<|eok|>"
                                 + txt_list[i] + "<|endoftext|>",
                                 truncation=True, 
                                 max_length=max_length, 
                                 padding="max_length",
                                 )
      prompts = tokenizer("<|startofprompt|>" + titles[i] + "<|endofprompt|>" 
                          + verdicts[i] + "<|endofverdict|>" + keywords[i] 
                          + "<|eok|>", truncation=True, max_length=max_length)
      
      """
      Each iteration then appends either the encoded tensor to a list,
      or the attention mask for that encoding to a list. The attention mask is
      a binary list of 1's or 0's which determine whether the langauge model
      should take that token into consideration or not. 
      """
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
      self.labels.append(torch.tensor(prompts["input_ids"]))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.labels[idx], self.attn_masks[idx] 

In [ ]:
from torch.utils.data import random_split

dataset = AitaDataset(titles, verdicts, stories, tokenizer, max_length=800)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {train_size} samples for training, and {val_size} samples for validation testing'

'There are 22500 samples for training, and 2500 samples for validation testing'

In [ ]:
train_dataset[0][0].shape

torch.Size([800])

In [ ]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# This step is optional but will enable reproducible runs.
seed_val = 67

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
epochs = 4
warmup_steps = 1e2
sample_every = 500

''' Initializing Optimizer '''
from transformers import AdamW
# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters. 
optimizer = AdamW(model.parameters(),
                  lr = 5e-4,
                  eps = 1e-8
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # Sampling for training is random
            batch_size = bs
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # Sampling for validation is sequential as the order doesn't matter.
            batch_size = bs
        )

In [ ]:
from transformers import get_linear_schedule_with_warmup

"""
Total training steps is the number of data points, times the number of epochs. 
Essentially, epochs are training cycles, how many times each point will be seen by the model. 
"""

total_steps = len(train_dataloader) * epochs

"""
set a variable learning rate which will help scan larger areas of the 
problem space at higher LR earlier, then fine tune to find the exact model minima 
at lower LR later in training.
"""
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
import random
import time
import datetime

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

total_t0 = time.time()

training_stats = []

model = model.to(device)

Beginning epoch 1 of 4


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batch 500 of 22500. Loss:1.8649296760559082. Time:0:02:34
Example output: 6 bestbest,best,aita,aita <|eok|> feel free to ask me what's going on. am i not an asshole? (aside from the fact that i don't like all the shit out there.) i'm a fairly successful writer (we've both had best,best <|eok|> worse,best writing). i've always believed in one thing: my work is <|endofverdict|>  best; it's not something we're constantly trying to present or the fact that we always say we like the things we do, like the things we do.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batch 1000 of 22500. Loss:1.5061593055725098. Time:0:05:17
Example output: wibta if my aunt (letting my bf have the "funeral"? if she's still alive) decides to help out? <|endofprompt|> asshole <|endofverdict|> funeral,wibta,aita,wibta,wibta,mother <|eok|> this post was really bad. i'm sorry i'm so bad, but i think we should at least address this in some sort of way? 

she was a pretty good aunt to me, but had been miserable. when she was dying she had to make another funeral for her. so a couple weeks before her death she had to tell her aunt she'd be coming home. a few weeks before her funeral she had the same problem. "let me keep in mind, i had a pretty bad time for her." she said "i wish i could get married", something which she didn't understand. i did feel bad because i would feel bad about that, but i didn't feel bad. so she was saying


...

Batch 13500 of 22500. Loss:2.8016884326934814. Time:1:14:40
Example output: aita for breaking a deal with a coworker? <|endofprompt|> not the asshole <|endofverdict|> aita,breaking,deal,coworker <|eok|> my coworker and i are still best friends but she hasn't noticed a new relationship with her (her) coworker. that's a story for another day.

two years ago i met her and we started talking about our plans, she was already planning to have her new relationship with me. we ended up having a couple dates and we decided to work it out.

i found out when she told me it was going to be a holiday thing and we ended up meeting up for a little bit. i noticed she texted me (i texted her at like midnight) and said that she hasn't seen a new relationship with me in several months, she's still on her last year. i asked why she was still on her first week of this new relationship, she asked why didn't she tell me sooner and i told her not to worry


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batch 14000 of 22500. Loss:3.193979024887085. Time:1:17:26
Example output: wibta for refusing to give up my seat <|endofprompt|> asshole <|endofverdict|> give,refusing,wibta,seat <|eok|> firstly, i am in a long-time relationship, and so is my boyfriend, and we share the same seat. my boyfriend and i sit for the same reasons:

1. we are both a full-time student, but he is a full-time college student. his family lives an hour away, so he works more often, and he spends a lot of time in his office/office building. 

2. when he's tired in the afternoon/early hours, he'll drop the bag, and that's the end of the day. my boyfriend, normally a big guy and always the same height as me, would be fine with sitting in my seat. i would be totally fine with his attention, so i could sit in his (unpackaged) space, even though i just don't get to sleep comfortably enough for


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batch 14500 of 22500. Loss:1.511335849761963. Time:1:20:11
Example output: aita for being upset with my boyfriend for making a joke about someone's appearance when he got engaged <|endofprompt|> not the asshole <|endofverdict|> got engaged,joke,aita,upset,boyfriend,fiance,actually,looking <|eok|> so, my boyfriend is from the usa, he's from england. for most of the time he's been dating this girl who i've been dating for a year. the problem is, i met him because he's from england with a really small and nice blue shirt. he is now in another england city where all the girls in england are white and they all like him. a while ago he and i were out of the same place, a little while after we got back home and we headed home. i've tried to be friendly with his friends, but they are just saying he's just joking and making fun of the people in england. 

i told him that he's making fun of the people in my


## TRAINING CELL | DONT RUN FOR GENERATION |

In [ ]:
for epoch_i in range(0, epochs):

    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_title = batch[1].to(device)
        b_masks = batch[2].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every 500 batches.
        if step % sample_every == 0 and not step == 0:

            PATH = "/content/drive/MyDrive/NLP_Project/modelcheckpoints/keywords_model_1.pt"
            torch.save({
                        'step': step,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        }, PATH)    
                
            elapsed = format_time(time.time() - t0)
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')

            model.eval()
            # SAVING MODEL CHECKPOINT EVERY 500 STEPS
            sample_outputs = model.generate(
                                    input = b_title,
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print(f'Example output: {tokenizer.decode(sample_output, skip_special_tokens=True)}')
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[2].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids,  
                             attention_mask = b_masks,
                             labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')